09 jul 2022
nrobot

Scrape and parse ampreviews forums

In [1]:
import argparse
import os
import re
import sys
import time
import requests
from urllib.parse import urlparse
import bs4
from bs4 import BeautifulSoup
from bs4.element import NavigableString
#bs4.element.NavigableString
import pandas as pd 
#from convert import shortToBytes, bytesToShort

In [2]:
#https://www.ampreviews.net/index.php?forums/discussion-new-york-city-manhattan.70/
example_url = 'https://www.ampreviews.net/index.php?threads/people-with-rubmaps-accounts.9374/'
response = requests.get(example_url, cookies=None, headers=None, timeout=10)

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')

# Scraper Code 

In [4]:
QUOTE_CSS_CLASS = 'div.bbCodeBlock--quote'

def extract_ampreviews_data(url):
    response = requests.get(url, cookies=None, headers=None, timeout=10)
    soup = BeautifulSoup(response.text, 'html.parser')
    msgs = soup.select('article.message--post')
    print('extracting reviews for url: ', url, f'found {len(msgs)} messages')
    data = []
    for msg in msgs:
        username = msg['data-author']
        user_id = msg.select_one('a.username')['data-user-id']
        post_id = msg['data-content'] # this is the msg id within the thread
        msg_time = msg.select_one('div.message-date time')['title']
        msg_epoch_time = msg.select_one('div.message-date time')['data-time']
        #msg_time  = msg.select('time')[1] # the msg timestamp is 2nd time (first is when author joined forum)

        msg_body = msg.select_one('article.message-body')

        if msg_body.select_one(QUOTE_CSS_CLASS):
            quote_src_postid = msg_body.select_one('a.bbCodeBlock-sourceJump')['href']
            quoted_text = msg_body.select_one('div.bbCodeBlock-expandContent').text
            # actual message contained in top level div
            msg_text = msg_body.find('div').contents
            msg_text= ''.join([x for x in msg_body.find('div').contents if isinstance(x, bs4.element.NavigableString)])
        else:
            quote_src_postid = None
            quoted_text = None
            msg_text = msg_body.text
        data.append( [username, user_id, post_id, msg_time, msg_text, quote_src_postid, quoted_text] )
    df = pd.DataFrame()
    if len(msgs) > 0:
        df = pd.DataFrame(data)
        df.columns = ['username', 'user_id', 'post_id', 'msg_time', 'msg_text', 'quote_src_postid', 'quoted_text']
        print('done extracting reviews for url: ', url, f' returning dataframe of shape {df.shape}')
        #print(df.username)
    return df

In [6]:
data = extract_ampreviews_data(example_url)

extracting reviews for url:  https://www.ampreviews.net/index.php?threads/people-with-rubmaps-accounts.9374/ found 20 messages
done extracting reviews for url:  https://www.ampreviews.net/index.php?threads/people-with-rubmaps-accounts.9374/  returning dataframe of shape (20, 7)


# Experiment 1: Run on list of urls 

In [7]:
#https://www.ampreviews.net/index.php?forums/discussion-new-york-city-manhattan.70,
#https://www.ampreviews.net/index.php?forums/boardwide-general-topics.61,

In [2]:
url_list = pd.read_csv('./list_urls.txt')
url_list.columns=['url','']
for url in url_list.to_numpy():
    print(url)

NameError: name 'pd' is not defined

In [9]:
time.sleep(0.01) # sleep 10 msec
print(time.ctime())

Sun Oct 30 20:41:12 2022


In [10]:
list_data = []
print(time.ctime())
for i, url in enumerate(url_list['url']):
    print(i, '/',  len(url_list), ',',  url)
    #if i > 4:
       #break
    data =  extract_ampreviews_data(url)
    if len(data) != 0:
        list_data.append(data)
    time.sleep(0.1) 
print(time.ctime())

Sun Oct 30 20:41:12 2022
0 / 40 , https://www.ampreviews.net/index.php?threads/review-hns-linda.116676/
extracting reviews for url:  https://www.ampreviews.net/index.php?threads/review-hns-linda.116676/ found 13 messages
done extracting reviews for url:  https://www.ampreviews.net/index.php?threads/review-hns-linda.116676/  returning dataframe of shape (13, 7)
1 / 40 , https://www.ampreviews.net/index.php?threads/lks-lsc-bgc-pick-one-from-each-org.116680/
extracting reviews for url:  https://www.ampreviews.net/index.php?threads/lks-lsc-bgc-pick-one-from-each-org.116680/ found 11 messages
done extracting reviews for url:  https://www.ampreviews.net/index.php?threads/lks-lsc-bgc-pick-one-from-each-org.116680/  returning dataframe of shape (11, 7)
2 / 40 , https://www.ampreviews.net/index.php?threads/review-mj-spa.116611/
extracting reviews for url:  https://www.ampreviews.net/index.php?threads/review-mj-spa.116611/ found 5 messages
done extracting reviews for url:  https://www.ampreviews

In [11]:
list_data

[         username user_id      post_id                  msg_time  \
 0       Hmmmrub45   11274  post-845721  Jul 11, 2022 at 10:28 AM   
 1      squeezebox   95751  post-845742  Jul 11, 2022 at 11:05 AM   
 2     AMPRomantic  159577  post-845768  Jul 11, 2022 at 11:59 AM   
 3          Sylvan   22210  post-846218  Jul 12, 2022 at 12:52 AM   
 4        Surfer12  102758  post-846559   Jul 12, 2022 at 2:09 PM   
 5        Jefftech  161106  post-846618   Jul 12, 2022 at 3:06 PM   
 6           이 회장님   56268  post-846803   Jul 12, 2022 at 7:35 PM   
 7   Itis_whatitis  140045  post-846817   Jul 12, 2022 at 7:52 PM   
 8      Ampmessiah  176432  post-848511  Jul 15, 2022 at 11:46 AM   
 9      Ampmessiah  176432  post-848570   Jul 15, 2022 at 1:00 PM   
 10       Surfer12  102758  post-848605   Jul 15, 2022 at 1:31 PM   
 11     Ampmessiah  176432  post-848640   Jul 15, 2022 at 2:08 PM   
 12          이 회장님   56268  post-848661   Jul 15, 2022 at 2:32 PM   
 
                                

In [12]:
forum_data = pd.concat(list_data)

In [13]:
forum_data.describe()

,username,user_id,post_id,msg_time,msg_text,quote_src_postid,quoted_text
count,369,369,369,369,369,123,123
unique,199,199,369,359,363,108,108
top,AutomaticSlim,18878,post-845721,"Jul 11, 2022 at 10:28 AM",\n\n\n\n\n\n \n,/index.php?goto/post&id=848511,\n\t\t\tRiding around loose in the back of a p...
freq,16,16,1,2,5,2,3


In [14]:
import joblib
joblib.dump(forum_data, './ampreviews_scraped_forum_dataframe.joblib')

['./ampreviews_scraped_forum_dataframe.joblib']

In [15]:
df2 = joblib.load('./ampreviews_scraped_forum_dataframe.joblib')
df2

,username,user_id,post_id,msg_time,msg_text,quote_src_postid,quoted_text
0,Hmmmrub45,11274,post-845721,"Jul 11, 2022 at 10:28 AM","\nTitle: Review: HNS LInda\nDate: Jul 10, 2022...",None,None
1,squeezebox,95751,post-845742,"Jul 11, 2022 at 11:05 AM","\nOh lordy, how the mighty have fallen...\n \n",None,None
2,AMPRomantic,159577,post-845768,"Jul 11, 2022 at 11:59 AM",\nSucks\n \n,None,None
3,Sylvan,22210,post-846218,"Jul 12, 2022 at 12:52 AM","\nOuch, I feel your pain. I will make sure t...",None,None
4,Surfer12,102758,post-846559,"Jul 12, 2022 at 2:09 PM",\nSorry to hear this. Previously a great place...,None,None
...,...,...,...,...,...,...,...
15,Edmond D.,112376,post-856946,"Jul 27, 2022 at 11:19 PM","Really?\n\nI mean, if and when I go to the DR,...",/index.php?goto/post&id=856940,\n\t\t\tI recommend staying at a hotel that re...
16,이 회장님,56268,post-856948,"Jul 27, 2022 at 11:27 PM",The concierge knows what’s going on and they a...,/index.php?goto/post&id=856946,"\n\t\t\tReally?\n\nI mean, if and when I go to..."
17,Edmond D.,112376,post-856953,"Jul 27, 2022 at 11:32 PM","\nSorry if I come off NAIVE, but how could a G...",None,None
18,이 회장님,56268,post-856958,"Jul 27, 2022 at 11:51 PM",Some girls will purposely create a fuss such a...,/index.php?goto/post&id=856953,"\n\t\t\tSorry if I come off NAIVE, but how cou..."


In [16]:
forum_data.to_csv('./ampreviews_scraped_forum_data.csv')

# Scratch

- username - post title - post date - (quoted text) - post text - url - date scraped
- list of dictionaries maybe?
- authros.append()

DataFrame[
    list_of_ids, list_of_post_text, list_of_usernames
]
DataFrame.to_csv()


post id, post date, poste_text, username
post384, feb 2019, i saw this, aadvarak



corpus = my_data[['post_text']]

nlp = convert+to+token(corpus)



In [17]:
msg = msgs[5]
quoted = msg.select_one('article.message-body').select_one('div.bbCodeBlock') is not None
print(quoted)
quoted = msg.select_one('article.message-body').select_one('div.bbCodeBlock')
if quoted:
    print('hi')
quoted.find('div')
noquote_text = ''.join([x for x in quoted.contents if isinstance(x, bs4.element.NavigableString)])
noquote_text

NameError: name 'msgs' is not defined

In [ ]:
quote = msg.select_one('article.message-body')\
    .select_one('div.bbCodeBlock')
quote

<div class="bbCodeBlock bbCodeBlock--expandable bbCodeBlock--quote">
<div class="bbCodeBlock-title">
<a class="bbCodeBlock-sourceJump" data-content-selector="#post-57426" data-xf-click="attribution" href="/index.php?goto/post&amp;id=57426">NYCSCorpio2015 said:</a>
</div>
<div class="bbCodeBlock-content">
<div class="bbCodeBlock-expandContent">
			Also tried the upload photos of the spas that don't have pics for 1 month credit.  Uploaded some and they were all not approved for no good reason.<br/>
<br/>
Disappointing site.
		</div>
<div class="bbCodeBlock-expandLink"><a>Click to expand...</a></div>
</div>
</div>